In [91]:
import numpy as np
from strategy import ProbStrategy
from random import randint

In [1]:
def takingStock(values):
    counts = [0]*7
    locs = [0]*7
    for i, value in enumerate(values):
        counts[value] += 1
        if locs[value] == 0:
            locs[value] = [i]
        else:
            locs[value] = locs[value] + [i]
    return counts, locs

In [101]:
def probTwoRolls(N):
    p = 0
    for k in range(N+1):
        p += 6**(N-k)*(5**k)/(6**(2*N))
    return p

def probOneRoll(N):
    return 1/(6**N)

    def prob(N, roll):
        if roll == 1:
            p = 0
            for k in range(N+1):
                p += 6**(N-k)*(5**k)/(6**(2*N))
            return p
        elif roll == 2:
            return 1/(6**N)

In [4]:
print(counts, locs)

[0, 2, 1, 0, 1, 1, 0] [0, [0, 4], [3], 0, [1], [2], 0]


In [18]:
keep = locs[np.argmax(counts)]
reroll = [i for i in range(5) if i not in keep]

[1, 2, 3]

In [30]:
values = [1, 1, 1, 3, 2]
counts, locs = takingStock(values)

In [99]:
def score(values):
    """
    Automatically scores the roll. Outputs a string of the name of the roll,
    and the money amount assigned to the roll.
    """
    counts = [0]*7
    for value in values:
        counts[value] += 1

    if 5 in counts:
        return 30
    elif 4 in counts:
        return 15
    elif (3 in counts) and (2 in counts):
        return 12
    elif 3 in counts:
        return 8
    elif not (2 in counts) and (counts[1] == 0 or counts[6] == 0):
        return 20
    elif counts.count(2) == 2:
        return 5
    else:
        return 0

In [95]:
keep = [1,2,3,4,5]
[randint(1,6) for i in range(5-len(keep))]

[]

In [124]:
values = [1, 2, 4, 4, 5]
keep = [2, 3]

score([values[i] for i in keep] + [randint(1,6) for i in range(5-len(keep))])

8

In [133]:
def fiveOfAKind(values, roll):
    counts, locs = takingStock(values)
    keep = locs[np.argmax(counts)]
    reroll = [i for i in range(5) if i not in keep]
    
    # Calculate expected value
    ex = 2*score([values[i] for i in keep] + [randint(1,6) for i in range(5-len(keep))]) + prob(len(reroll), roll)*scores['Five of a Kind']
    return reroll, ex

def fourOfAKind(values, roll):
    counts, locs = takingStock(values)
    safe = np.max(counts)
    if safe >= 4:
        keep = [0, 1, 2, 3, 4]
        reroll = []
    else:
        keep = locs[np.argmax(counts)]
        reroll = [i for i in range(5) if i not in keep]
    ex = 1.5*score([values[i] for i in keep] + [randint(1,6) for i in range(5-len(keep))]) + prob(len(reroll), roll)*scores['Four of a Kind']
    return reroll, ex
    
    
def threeOfAKind(values, roll):
    counts, locs = takingStock(values)
    safe = np.max(counts)
    if safe >= 3:
        keep = [0, 1, 2, 3, 4]
        reroll = []
    else:
        keep = locs[np.argmax(counts)]
        reroll = [i for i in range(5) if i not in keep]
    ex = 1.5*score([values[i] for i in keep] + [randint(1,6) for i in range(5-len(keep))]) + prob(len(reroll), roll)*scores['Three of a Kind']
    return reroll, ex
    
def fullHouse(values, roll):
    counts, locs = takingStock(values)
    safe = np.max(counts)
    if (3 in counts) & (2 in counts):
        keep = [0, 1, 2, 3, 4]
        reroll = []
    elif safe == 5:
        reroll = locs[counts.index(5)][0:2]
        keep = [i for i in range(5) if i not in reroll]
    elif safe == 4:
        reroll = locs[counts.index(1)] + locs[counts.index(4)][0:1]
        keep = [i for i in range(5) if i not in reroll]
    elif safe <= 3:
        keep = locs[np.argmax(counts)]
        reroll = [i for i in range(5) if i not in keep]
    ex = 1.5*score([values[i] for i in keep] + [randint(1,6) for i in range(5-len(keep))]) + prob(len(reroll), roll)*scores['Full House']
    return reroll, ex
    
def straight(values, roll):
    counts, locs = takingStock(values)
    keep = []
    for i in [2,3,4,5]:
        if i in values:
            keep.append(values.index(i))
    if (1 in values) & (6 in values):
        keep.append(values.index(1))
    elif 1 in values:
        keep.append(values.index(1))
    elif 6 in values:
        keep.append(values.index(6))
    reroll = [i for i in range(5) if i not in keep]
    ex = 1.5*score([values[i] for i in keep] + [randint(1,6) for i in range(5-len(keep))]) + prob(len(reroll), roll)*scores['Straight']
    return reroll, ex
    
    
def twoPairs(values, roll):
    counts, locs = takingStock(values)
    safe = np.max(counts)
    full, _ = fullHouse(values, 1)
    if (counts.count(2) == 2) | (full == []):
        reroll = []
        keep = [0, 1, 2, 3, 4]
    elif safe == 5:
        reroll = [0, 1]
        keep = [2, 3, 4]
    elif safe == 4:
        reroll = locs[counts.index(1)] + locs[counts.index(4)][0:2]
        keep = [i for i in range(5) if i not in reroll]
    elif safe == 3:
        keep = locs[counts.index(3)][0:2]
        reroll = [i for i in range(5) if i not in keep]
    elif safe == 2:
        keep = locs[counts.index(2)]
        reroll = [i for i in range(5) if i not in keep]
    else:
        reroll = [0,1,2,3,4]
        keep = []
    ex = 1.5*score([values[i] for i in keep] + [randint(1,6) for i in range(5-len(keep))]) + prob(len(reroll), roll)*scores['Two Pairs']
    return reroll, ex
    
    

In [154]:
values = [1, 1, 1, 1, 4]
fiveOfAKind(values, 1)

([4], 39.16666666666667)

In [155]:
rerolls = {'Five of a Kind': fiveOfAKind,
           'Four of a Kind': fourOfAKind,
           'Three of a Kind': threeOfAKind,
           'Full House': fullHouse,
           'Straight': straight,
           'Two Pairs': twoPairs}
scores = {'Five of a Kind': 30,
          'Four of a Kind': 15,
          'Three of a Kind': 8,
          'Full House': 12,
          'Straight': 20,
          'Two Pairs': 5}

In [156]:
expected = []
toRolls = []
roll = 1
for key, item in rerolls.items():
    reroll, ex = item(values, roll)
    expected.append(ex)
    toRolls.append(reroll)
    
toRolls[np.argmax(expected)]
expected

[69.16666666666667,
 37.5,
 30.5,
 23.34259259259259,
 0.2876371742112483,
 0.07190929355281207]

In [153]:
ProbStrategy([1,1,1,1,4], 1)

[]